--- 

# Les principes FAIR appliqués à la bioinformatique

**Outils et environnements FAIR pour la bioanalyse**

Mardi 15 décembre 2020

Thomas Denecker - *Data brokering* - IFB core

---

## Un notebook R interactif

### Chargement des packages

In [ ]:
options(warn=-1)
library(plotly)
library(DT)
library(dplyr)
library(plotly)

### Lectures du tableau de résultats de SARtootls

In [ ]:
dataSartools <- read.csv2("demo/tables/DEPLETEDvsSTANDARD.complete.txt", sep = "\t", header = T,stringsAsFactors = F)
dataSartools <- dataSartools %>% 
filter(log2FoldChange != "") %>% 
arrange(desc(log2FoldChange))%>% 
mutate(log2FoldChange = as.numeric(log2FoldChange), 
      pvalue = as.numeric(pvalue))

### Datatable 

Documentation : https://rstudio.github.io/DT/

In [ ]:
datatable(dataSartools[, c("Id", "log2FoldChange", "pvalue")], extensions = 'Buttons', filter = 'top', 
          options = list(
            dom = 'Bfrtip',
            buttons = c('copy', 'csv', 'excel', 'pdf', 'print'), 
            pageLength = 5, autoWidth = TRUE
          ))

### Volcano plot avec Plotly

Documentation : https://plotly.com/r/

In [ ]:
inter = cbind(x = dataSartools$log2FoldChange,
              y = -log10(dataSartools$pvalue), 
              pvalue = dataSartools$pvalue, 
              feature = dataSartools$Id)
inter = na.omit(inter)
inter = as.data.frame(inter)
inter[,1] = as.numeric(as.character(inter[,1]))
inter[,2] = as.numeric(as.character(inter[,2]))
    
color = rep("black", nrow(inter))
pos = which(abs(inter$x) >= 2 & inter$y >= -log10(0.01))
color[pos] = "red"
    
plot_ly(inter, x = ~x, y = ~y, type = 'scatter', mode = 'markers',
            text = ~paste("Feature: ", feature, '<br>logFC:', x, '<br>Pvalue:', pvalue ),
            marker = list(color = color)) %>%
      layout(title = 'Volcano plot',
             shapes=list(list(type='line', x0=min(inter$x)-1, x1= max(inter$x)+1, y0=-log10(0.01), y1=-log10(0.01), line=list(dash='dot', width=1)),
                         list(type='line', x0=-2, x1= -2, y0=0, y1=max(inter$y), line=list(dash='dot', width=1)),
                         list(type='line', x0=2, x1= 2, y0=0, y1=max(inter$y), line=list(dash='dot', width=1))),
             yaxis = list(zeroline = FALSE, title= "-log10(adjusted pvalue)"),
             xaxis = list(zeroline = FALSE, title= "log2(fold change)"))

In [ ]:
sessionInfo()